In [1]:
import os
import sys
import json
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
#exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

In [2]:
from pyspark import SparkContext, SparkConf

conf = SparkConf()
conf.set("spark.app.name", "karavaev_andrei_spark_lab_1") 

sc = SparkContext(conf=conf)

In [3]:
from collections import Counter

In [4]:
FILM_ID = '22'

In [5]:
sc

<SparkContext master=yarn appName=karavaev_andrei_spark_lab_1>

In [6]:
!hdfs dfs -ls 

Found 1 items
drwxr-xr-x   - andrey.karavaev andrey.karavaev          0 2021-02-28 17:10 .sparkStaging


In [7]:
rdd_film_data = sc.textFile("/labs/laba01/ml-100k/u.data")
rdd_film_data_cached = rdd_film_data.cache()

In [8]:
#rdd_item_data = sc.textFile("/labs/laba01/ml-100k/u.item")
# вообще нам эти данные для выполнения задания не нужны

In [9]:
rdd_film_data_cached.getNumPartitions()

2

In [10]:
rdd_film_data_cached = rdd_film_data_cached.map(lambda x: x.split("\t"))

In [11]:
rdd_film_data_cached.take(10)

[['196', '242', '3', '881250949'],
 ['186', '302', '3', '891717742'],
 ['22', '377', '1', '878887116'],
 ['244', '51', '2', '880606923'],
 ['166', '346', '1', '886397596'],
 ['298', '474', '4', '884182806'],
 ['115', '265', '2', '881171488'],
 ['253', '465', '5', '891628467'],
 ['305', '451', '3', '886324817'],
 ['6', '86', '3', '883603013']]

In [12]:
rdd_film_data_cached = rdd_film_data_cached.map(lambda x: x[1:3])

## Сделал 3 разными способами (самый каноничный наверное последний)

## Counter approach

In [128]:
scores_target = rdd_film_data_cached.groupByKey().map(lambda x: (x[0], Counter(x[1]))).filter(lambda x: x[0] == FILM_ID).collect()
scores_target = scores_target[0][1]

In [129]:
counter_sums = rdd_film_data_cached.groupByKey().map(lambda x: (x[0], Counter(x[1]))).values().collect()
counter_sum = sum(counter_sums, Counter())

In [130]:
scores_target = [scores_target[i] for i in sorted(scores_target)]

In [131]:
scores_all = [counter_sum[i] for i in sorted(counter_sum)]

In [132]:
scores_target

[5, 14, 46, 98, 134]

In [133]:
scores_all

[6110, 11370, 27145, 34174, 21201]

In [134]:
scores_output = {"hist_film":scores_target, "hist_all":scores_all}

In [135]:
scores_output

{'hist_film': [5, 14, 46, 98, 134],
 'hist_all': [6110, 11370, 27145, 34174, 21201]}

## UDF(??) approach

In [151]:
def count_fixed_values(iterable):
    value_counts = [0 for i in range(5)]
    for value in iterable:
        value = int(value) - 1
        value_counts[value] += 1
    return value_counts

def sum_values_in_list(iter_1, iter_2):
    for num, elem in enumerate(iter_2):
        iter_1[num] += elem
    return iter_1

In [145]:
scores_target = rdd_film_data_cached.groupByKey().map(lambda x: (x[0], count_fixed_values(x[1]))).filter(lambda x: x[0] == FILM_ID).collect()
scores_target = scores_target[0][1]

In [146]:
scores_target

[5, 14, 46, 98, 134]

In [153]:
scores_all = rdd_film_data_cached.groupByKey().map(lambda x: (x[0], count_fixed_values(x[1]))).values().reduce(sum_values_in_list)

In [154]:
scores_all

[6110, 11370, 27145, 34174, 21201]

In [ ]:
scores_output = {"hist_film":scores_target, "hist_all":scores_all}

## Groupby count?? aproach

In [47]:
scores_target = rdd_film_data_cached.filter(lambda x: x[0] == FILM_ID) \
                                 .groupBy(lambda x: x[1]) \
                                 .map(lambda x: (x[0], len(x[1]))) \
                                 .sortByKey() \
                                 .values() \
                                 .collect()

In [48]:
scores_target

[5, 14, 46, 98, 134]

In [49]:
scores_all = rdd_film_data_cached.groupBy(lambda x: x[1]) \
                                    .map(lambda x: (x[0], len(x[1]))) \
                                    .sortByKey() \
                                    .values() \
                                    .collect()

In [50]:
scores_all

[6110, 11370, 27145, 34174, 21201]

In [52]:
scores_output = {"hist_film":scores_target, "hist_all":scores_all}

In [53]:
scores_output

{'hist_film': [5, 14, 46, 98, 134],
 'hist_all': [6110, 11370, 27145, 34174, 21201]}

## STOP CONTEXT

In [51]:
sc.stop()

## Dumping results

In [104]:
json.dumps(scores_output)

'{"hist_film": [5, 14, 46, 98, 134], "hist_all": [6110, 11370, 27145, 34174, 21201]}'

In [105]:
!pwd

/data/home/andrey.karavaev


In [111]:
with open("./lab01.json", mode='w') as file:
    json.dump(scores_output, file)

In [112]:
!cat lab01.json

{"hist_film": [5, 14, 46, 98, 134], "hist_all": [6110, 11370, 27145, 34174, 21201]}